# Priority queue

Main point: like a stack, but ordered, in the sense that we put things in randomly, but pop() them always in order, from high priority to low priority. If we put() it only once, and popped it only once, we would have just sorted the thing, but we want something to which we can add stuff as it arises, and yet still always pop() the top priority element, at all times.

I will be implementing it so that high priotity = later letters (z has higher priority over a) as that's what textbooks seem to do in most cases. In this case the parent node is always larger (later in the alphabet) than its children.

The 0th (aka first for normal people) element is kept empty, which seems stupid until you realize that this way the children of element i lie exactly at 2i and 2i+1.

In [1]:
# Annotated version

class pq():
    """Max priority queue."""
    def __init__(self, init=[]):
        self.x = [0]
        self.last = 0
        for i in init:
            self.add(i)
        
    def __str__(self):
        #return ' '.join(str(self.x))
        return str(self.x)
    
    def feed(self,a):
        """Add more than one element at once, from either str or array."""
        for i in a:
            self.add(i)
    
    def add(self,a):
        self.x.append(a)
        self.last += 1
        self._swim(self.last)
    
    def swap(self,i,j):
        (self.x[i] , self.x[j]) = (self.x[j] , self.x[i])
        
    def pop(self,n=1):
        if n>1:                                            # Not a part of the algo; I just want to automate testing
            return ''.join([self.pop() for i in range(min(n,self.last))]) # By removing more than 1 element at once
        out = self.x[1]
        self.swap(1,self.last) # As we cannot have holes in the tree, first swap the doomed element wiht the tail
        del self.x[self.last]  # Now remove it and clean up the mess. (Isn't it strange that del isn't a method?)
        self.last -= 1
        self._sink(1)       # restore tree integrity (as now we have a small element on top)
        return out
    
    def _sink(self,i):
        while i*2 <= self.last:
            j = i*2                                       # First consider left kid.
            if (j<self.last) and (self.x[j]<self.x[j+1]): # If this node has 2 children, and left < right,
                j += 1                                    # ..switch to the right one (larger one)
            if (self.x[j]<=self.x[i]): # If current is larger than the largest kid: no need to go further
                break
            self.swap(i,j) # Current node is smaller than its largest kid, so sink it
            i = j
        
    def _swim(self,i):
        while i>1 and self.x[i//2]<self.x[i]: # While parent exist, and it's smaller than you, ascend.
            self.swap(i//2,i)
            i //= 2
            
    def visualize(self,i=1,level=1):
        # Depth-first traversal of the binary tree
        for _ in range(level-1): 
            print('|',end='')
        print(self.x[i])
        if i*2 <= self.last:
            self.visualize(i*2,level+1)
        if i*2+1 <= self.last:
            self.visualize(i*2+1,level+1)
            
        
a = pq(list('toxic carrot'))
print(a)
a.add('d')
print(a)
print(a.pop(4))
print(a)
print(a.pop(4))
print(a)
a.visualize()
a.feed('lazy parrots are spherical')
print(a)
print(a.pop(9))
print(a)
print(a.pop(100))
print(a)

[0, 'x', 't', 't', 'r', 'r', 'c', 'a', 'i', 'o', 'c', 'o']


AttributeError: 'pq' object has no attribute 'insert'

### Notes:

* We don't know for sure whether the left or the right branch has a smaller value, so we have to check it while swimming down. I'm guessing one could also order them at construction phase (while swimming up), which would have simplified swimming down. But probably not much of a difference here.
* IRL in languages with memory allocation you wouldn't want to change array size with every deletion; instead you'd do stardard "array resize" if it gets too tight (next element doesn't fit) or too loose (say, more than half of the array is empty).
* Terniary (and higher order) trees are apparently also used in practice. Not sure why somebody would do it though. What's the benefit? Not clear.
* May be used for sorting (**heapsort**; apparently runs in 2(n*lg(n) + n). Two tricks here: 
 1. instead of repeatedly running insert(), it's better to just place the list there, and then "heapify" it by gowing through all nodes with kids (last//2 and lower), right-to-left, "sinking" (swimdown) if necessary
 2. then we repeatedly do pop(), and place former max instead of the former last element (so heap scrolls to the left, while sorted array replaces it to the right, all in-place)